In [1]:
import pathlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import guitarpro as pygp
import glob

path_to_general_read_folder = "..\..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\..\data\BGTG\BGTG-v0.1"

In [2]:
def ultimate_get_filename(row, series_filenames, series_paths):
    name = row['Fichier']
    name = name.split('/')[-1]
    name = name.split('.')[0]
    splits = name.split('-')
    current_name = splits[0]
    filenames_list = series_filenames.values
    i = 1
    while current_name not in filenames_list:
        # print(current_name)
        try:
            current_name += '-' + splits[i]
            i += 1
        except:
            row['Dadagp_Path'] = 'error'
            row['File_Name'] = 'error'
            return row
    
    indexes = series_filenames[series_filenames == current_name].index
    paths = series_paths[indexes].values
    row['Dadagp_Path'] = paths[0]
    row['File_Name'] = current_name
    return row
    
    
def ultimate_get_filename_2(row, series_filenames, series_paths):
    name = row['Fichier']
    name = name.split('/')[-1]
    name = name.split('.')[0]
    splits = name.split('-')
    current_name = splits[0]
    i = 1
    while current_name not in series_filenames.values:
        # print(current_name)
        try:
            current_name += '-' + splits[i]
            i += 1
        except:
            row['Dadagp_Path'] = 'error'
            row['File_Name'] = 'error'
            return row
    
    row['File_Name'] = current_name
    indexes = series_filenames[series_filenames == current_name].index
    paths = series_paths[indexes].values
    for path in paths:
        row_part = row['Partie']
        gp_file = pygp.parse(str(path))
        file_parts = [track.name for track in gp_file.tracks]
        if row_part in file_parts:
            row['Dadagp_Path'] = path

    return row

In [10]:
df_rg = pd.read_csv('..\\..\\data\\function_guitar-ext-with-predictions.csv')
df_rg = df_rg[['Fichier', 'Track_Name', 'Partie', 'Instrument', 'Mesure', 'rg-estimation']]
display(df_rg.head())
df_rg.shape

,Fichier,Track_Name,Partie,Instrument,Mesure,rg-estimation
0,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07
1,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,2,4.580104e-07
2,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,3,6.601997e-06
3,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,4,4.111985e-04
4,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,5,4.352387e-05


(4165789, 6)

In [7]:
DADAGPPATH = pathlib.Path(path_to_general_read_folder)

all_paths = list(DADAGPPATH.rglob('**/*.pygp.gp5'))
# retrieve ALL filenames in DadaGP 
all_filenames = [f.name for f in all_paths]
all_filenames_no_ext = [f.name[:-13] for f in all_paths]

index = np.random.randint(0, len(all_paths))

all_paths[index], all_filenames[index], all_filenames_no_ext[index]
series_paths = pd.Series(all_paths)
series_filenames = pd.Series(all_filenames_no_ext)

In [12]:
tqdm.pandas()
# df.reset_index(drop=True, inplace=True)
# drop Unnamed: 0
df_rg_inst_part = df_rg.drop_duplicates(subset=['Fichier', 'Partie']).copy()
df_rg_inst_part.loc[:, 'File_Name'] = 'Not found'
df_rg_inst_part.loc[:, 'Dadagp_Path'] = 'Not found'
display(df_rg_inst_part.head())
df_rg_inst_part.shape


,Fichier,Track_Name,Partie,Instrument,Mesure,rg-estimation,File_Name,Dadagp_Path
0,../data/DadaGP8-gpif/ - Isolated.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07,Not found,Not found
131,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 1....,Tudo Que Vai\nCapital Inicial by Bifaum,Trilha 1,clean,2,3.238349e-05,Not found,Not found
341,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 2....,Tudo Que Vai\nCapital Inicial by Bifaum,Trilha 2,clean,2,9.926770e-01,Not found,Not found
386,../data/DadaGP8-gpif/*.gpif,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07,Not found,Not found
517,../data/DadaGP8-gpif/1 God - Grace.gpif,Grace\n1 God,Track 1,clean,1,2.090054e-08,Not found,Not found


(56574, 8)

In [ ]:
# Count number of errors
display(df_rg[df_rg['Dadagp_Path'] == 'error']['Fichier'].unique())
# Windows doesn't accept files starting with a blank space, we have to rename those

# Remove the duplicates from the dataframe
df_rg = df_rg[df_rg['Dadagp_Path'] != 'error']

array(['../data/DadaGP8-gpif/*.gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (1).gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (2).gpif',
       '../data/DadaGP8-gpif/Beethoven - Moonlight Sonata (3) (3).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (1).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (2).gpif',
       '../data/DadaGP8-gpif/Cambefort, JP - Blues Pour Francois (3).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (1).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (2).gpif',
       '../data/DadaGP8-gpif/Eminem - Lose Yourself (Intro) (3).gpif',
       '../data/DadaGP8-gpif/Irish - Kingfair (bass Solo Version) (1).gpif',
       '../data/DadaGP8-gpif/Joel, Billy - Just the way you are (1).gpif',
       '../data/DadaGP8-gpif/Kaukonen, Jorma - Hesitation Blues (1).gpif',
       '../data/DadaGP8-gpif/Saez - MontCe LO Haut (2) (1).gpif',
       '.

In [ ]:
total_df_track = pd.DataFrame()
i=0

# For each track we add the DadaGP_name column
for index, row in tqdm(df_rg_inst_part.iterrows()):
    
    # STEP 1: FIND FILENAME AND DADAGP_PATH
    name = row['Fichier']
    name = name.split('/')[-1]
    name = name.split('.')[0]
    splits = name.split('-')
    current_name = splits[0]
    i = 1
    error = False
    while current_name not in series_filenames.values:
        # print(current_name)
        try:
            current_name += '-' + splits[i]
            i += 1
        except:
            row['Dadagp_Path'] = 'error'
            row['File_Name'] = 'error'
            error = True
    
    if not error:
        row['File_Name'] = current_name
        indexes = series_filenames[series_filenames == current_name].index
        paths = series_paths[indexes].values
        for path in paths:
            row_part = row['Partie']
            gp_file = pygp.parse(str(path))
            file_parts = [track.name for track in gp_file.tracks]
            if row_part in file_parts:
                row['Dadagp_Path'] = path
                continue
    
        df_track = df_rg_inst_part[df_rg_inst_part['Dadagp_Path'] == path].copy() # We copy to avoid SettingWithCopyWarning
        
        dada_gp_path = df_track['Dadagp_Path'].values[0]
        gp_file = pygp.parse(dada_gp_path)
    
    # No need for this anymore normally as the path are retrieved from DadaGP
    # try:
    #     gp_file = pygp.parse(dada_gp_path)
    # except:
    #     print('Error with file:', dada_gp_path)
    #     continue
        
    inst_list = []
    for part in gp_file.tracks:
        part_name = part.name
        
        if part_name not in df_track['Partie'].values:
            continue
        inst_name = df_track[df_track['Partie'] == part_name]['Instrument'].values[0]
        inst_list.append((part_name, inst_name))
    
    # Create the Dadagp_Name column filled with 'not_filled_yet'
    df_track['Dadagp_Name'] = 'not_filled_yet'
    nb_clean = 0
    nb_distorted = 0
    for part_inst_tuple in inst_list:
        part = part_inst_tuple[0]
        inst = part_inst_tuple[1]
        if inst == 'clean':
            df_track.loc[df_track['Partie'] == part, 'Dadagp_Name'] = 'clean' + str(nb_clean)
            nb_clean += 1
        elif inst == 'distorted':
            df_track.loc[df_track['Partie'] == part, 'Dadagp_Name'] = 'distorted' + str(nb_distorted)
            nb_distorted += 1
        else:
            df_track.loc[df_track['Partie'] == part, 'Dadagp_Name'] = inst
            
    if 'not_filled_yet' in df_track['Dadagp_Name'].unique():
        display(df_track)
        print(inst_list)
    
    # Join df_track with df_inst_part
    total_df_track = pd.concat([total_df_track, df_track])
    i+=1
    if i == 100:
        break

df_inst_part.shape, total_df_track.shape

# The method is correct but takes a long time to compute (1h)

  0%|          | 28/24051 [00:10<1:41:31,  3.94it/s]

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation,Dadagp_Name
3843,../data/DadaGP8-gpif/3 Doors Down - Be Like Th...,..\..\data\DadaGP\DadaGP-v1.1\3\3 Doors Down\3...,3 Doors Down - Be Like That,Be Like that\n3 Doors Down,Acoustic guitar,clean,1,3.069821e-05,not_filled_yet
3919,../data/DadaGP8-gpif/3 Doors Down - Be Like Th...,..\..\data\DadaGP\DadaGP-v1.1\3\3 Doors Down\3...,3 Doors Down - Be Like That,Be Like that\n3 Doors Down,Bass,bass,13,5.398947e-09,not_filled_yet
3998,../data/DadaGP8-gpif/3 Doors Down - Be Like Th...,..\..\data\DadaGP\DadaGP-v1.1\3\3 Doors Down\3...,3 Doors Down - Be Like That,Be Like that\n3 Doors Down,Rhythm guitar 2,distorted,76,2.160044e-04,not_filled_yet
4014,../data/DadaGP8-gpif/3 Doors Down - Be Like Th...,..\..\data\DadaGP\DadaGP-v1.1\3\3 Doors Down\3...,3 Doors Down - Be Like That,Be Like that\n3 Doors Down,Rhythm guitar,distorted,13,8.582433e-06,not_filled_yet
4078,../data/DadaGP8-gpif/3 Doors Down - Be Like Th...,..\..\data\DadaGP\DadaGP-v1.1\3\3 Doors Down\3...,3 Doors Down - Be Like That,Be Like That\n3 Doors Down,Track 1,clean,1,7.606207e-06,clean0


[('Track 1', 'clean')]


  0%|          | 63/24051 [00:22<2:29:10,  2.68it/s]

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation,Dadagp_Name
10506,../data/DadaGP8-gpif/311 - Homebrew (2)-Nick.gpif,..\..\data\DadaGP\DadaGP-v1.1\3\311\311 - Home...,311 - Homebrew (2),Homebrew\n311,Nick,distorted,5,1.075907e-07,not_filled_yet
10575,../data/DadaGP8-gpif/311 - Homebrew (2)-P-Nut....,..\..\data\DadaGP\DadaGP-v1.1\3\311\311 - Home...,311 - Homebrew (2),Homebrew\n311,P-Nut,bass,1,1.939719e-13,not_filled_yet
10648,../data/DadaGP8-gpif/311 - Homebrew (2)-Tim.gpif,..\..\data\DadaGP\DadaGP-v1.1\3\311\311 - Home...,311 - Homebrew (2),Homebrew\n311,Tim,distorted,1,1.075907e-07,not_filled_yet
10721,../data/DadaGP8-gpif/311 - Homebrew (2)-Track ...,..\..\data\DadaGP\DadaGP-v1.1\3\311\311 - Home...,311 - Homebrew (2),None\nNone,Track 1,distorted,1,1.148877e-07,distorted0
10790,../data/DadaGP8-gpif/311 - Homebrew (2)-Track ...,..\..\data\DadaGP\DadaGP-v1.1\3\311\311 - Home...,311 - Homebrew (2),None\nNone,Track 2,distorted,5,3.390092e-06,distorted1


...

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation,Dadagp_Name
13077,../data/DadaGP8-gpif/38 Special - Like No Othe...,..\..\data\DadaGP\DadaGP-v1.1\3\38 Special\38 ...,38 Special - Like No Other Night,Like No Other Night\n38 Special,distorted,distorted,1,0.065534,not_filled_yet


  0%|          | 88/24051 [00:27<1:32:19,  4.33it/s]

[]


  0%|          | 99/24051 [00:31<2:09:00,  3.09it/s]


((56419, 8), (255, 9))